# Ensemble Time Series

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# List of Stocks and ETFs
Provided by Thomas Choi.

In [2]:
stock_list = pd.read_csv("../stocks_and_etfs/stock_list.csv")
etf_list = pd.read_csv("../stocks_and_etfs/etf_list.csv")

In [3]:
import random

# Choose a random stock
stock_symbol = random.choice(stock_list.iloc[:,0])
print(stock_symbol)

CCL


## MySQL connection
Choosing one stock from SQL query to reduce query time.

In [4]:
import os
import sys
import mysql.connector

HOST="143.244.188.157"
PORT="3306"
USER="patrick-finProj"
PASSWORD="Pat#21$rick"

try: 
    conn = mysql.connector.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database="GlobalMarketData"
    )
    query = f"SELECT Date, Close, Open, High, Low, Volume from histdailyprice3 WHERE Symbol='{stock_symbol}';"
    histdailyprice3 = pd.read_sql(query, conn)
    conn.close()
except Exception as e:
    conn.close()
    print(str(e))

# Data

In [5]:
df = histdailyprice3.copy()
df.head()

,Date,Close,Open,High,Low,Volume
0,1998-01-01,27.69,27.69,27.69,27.69,0
1,1998-01-02,27.75,27.81,27.00,27.19,554400
2,1998-01-05,27.22,27.91,27.00,27.50,787600
3,1998-01-06,27.31,27.50,27.00,27.00,436600
4,1998-01-07,27.03,27.78,27.03,27.75,621400


In [6]:
diff_stock_prices = df["Close"].diff().dropna()
diff_stock_prices.head()

1    0.06
2   -0.53
3    0.09
4   -0.28
5    0.59
Name: Close, dtype: float64

# Normalize

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(diff_stock_prices.values.reshape(-1,1))

# Train-Test Split

In [15]:
# Defining the number of steps for prediction
n_steps = 13
# Defining the number of features as 1
n_features = 1

In [12]:
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [16]:
split = int(len(diff_stock_prices.values) * 0.95)
diff_train = scaled_data[:split]
diff_test = scaled_data[split:]

X_train, y_train = split_sequence(diff_train, n_steps)
X_train = X_train.reshape((X_train.shape[0],  X_train.shape[1], n_features))

# LSTM

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

model = Sequential()
# Identifying the number of hidden neurons, the activation function,  and input shape
model.add(LSTM(512, activation='relu', 
               input_shape=(n_steps, n_features), 
               return_sequences=True))
# Adding a dropout layer to prevent overfitting
model.add(Dropout(0.2))
# Adding one more hidden layer with 256 neurons, with a relu activation function
model.add(LSTM(256,activation='relu'))
# Flattening the model to vectorize the three-dimensional matrix
model.add(Flatten())
# Adding an output layer with a linear activation function
model.add(Dense(1, activation='linear'))
# Compiling LSTM with Root Mean Square Propagation and mean squared error (MSE)
model.compile(optimizer='rmsprop', loss='mean_squared_error',
                       metrics=['mse'])

2022-01-06 11:07:40.448176: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=400, batch_size=150, verbose=0,
                    validation_split = 0.10)

2022-01-06 11:08:17.786098: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [ ]:
start = X_train[X_train.shape[0] - 13]
         x_input = start
         x_input = x_input.reshape((1, n_steps, n_features))